In [2]:
import torchvision
import torch
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from torchvision import transforms
from torchvision.io import read_image
from art.estimators.classification import PyTorchClassifier
# from art.attacks.evasion import ProjectedGradientDescentPyTorch
from torchattacks import PGD
from art.utils import random_sphere
from art.config import ART_NUMPY_DTYPE
import numpy as np
from torchvision.transforms import v2
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights, vgg19 , VGG19_Weights, resnet18, ResNet18_Weights, densenet121,  densenet121, DenseNet121_Weights, vit_b_16, ViT_B_16_Weights, efficientnet_b2, EfficientNet_B2_Weights,  swin_s, Swin_S_Weights, convnext_base, ConvNeXt_Base_Weights, squeezenet1_0, SqueezeNet1_0_Weights

/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

In [3]:
import os
os.environ['SLURM_JOBID']

'569949'

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
models = [("vit_b_16",vit_b_16, ViT_B_16_Weights),
          ("efficientnet_b0" , efficientnet_b0, EfficientNet_B0_Weights),
          ("swin_s", swin_s, Swin_S_Weights),
          ("densenet121" , densenet121, DenseNet121_Weights),
          ("resnet18" ,resnet18, ResNet18_Weights)]
            # ("convnext_base", convnext_base,ConvNeXt_Base_Weights)]
         # ("squeezenet1_0",squeezenet1_0, SqueezeNet1_0_Weights)]

In [6]:
class model_with_normalization(nn.Module):
    def __init__(self, model, normalization):
        super(model_with_normalization, self).__init__()
        self.model = model 
        self.normalization = normalization
        
    def forward(self, x):
        x = self.normalization(x)
        if x.shape == (3,224,224):
          x = x.unsqueeze(0)
        out = self.model(x)
        return out
    
normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

In [7]:
import os
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.data = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Initialize your custom dataset
# new_dataset = CustomDataset()

In [8]:
def read_image(image_path):
    return Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),])

In [ ]:
import pickle
import random
from torch.cuda.amp import autocast
trans = [
    transforms.RandomAffine(degrees=(-2, 2), translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=(224, 224), padding=10),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomAdjustSharpness(sharpness_factor=2,p=1),
    transforms.RandomAutocontrast(1)]


augmentations = transforms.Compose(trans)


In [14]:
e=12.75

In [15]:
"""Implementation of sample attack."""
import os
import torch
from torch.autograd import Variable as V
import torch.nn.functional as F
from torchvision import transforms as T
from torchvision.transforms import ToTensor, ToPILImage, transforms
from tqdm import tqdm
import numpy as np
from PIL import Image
# from Normalize import Normalize
# from loader import ImageNet
# from torch.utils.data import DataLoader
import argparse
# import pretrainedmodels

# parser = argparse.ArgumentParser()
# parser.add_argument('--input_csv', type=str, default='./dataset/images.csv', help='Input directory with images.')
# parser.add_argument('--input_dir', type=str, default='./dataset/images', help='Input directory with images.')
# parser.add_argument('--output_dir', type=str, default='./outputs/', help='Output directory with adversarial images.')
# parser.add_argument('--mean', type=float, default=np.array([0.5, 0.5, 0.5]), help='mean.')
# parser.add_argument('--std', type=float, default=np.array([0.5, 0.5, 0.5]), help='std.')
# parser.add_argument("--max_epsilon", type=float, default=16.0, help="Maximum size of adversarial perturbation.")
# parser.add_argument("--num_iter_set", type=int, default=10, help="Number of iterations.")
# parser.add_argument("--image_width", type=int, default=299, help="Width of each input images.")
# parser.add_argument("--image_height", type=int, default=299, help="Height of each input images.")
# parser.add_argument("--batch_size", type=int, default=20, help="How many images process at one time.")
# parser.add_argument("--momentum", type=float, default=1.0, help="Momentum")
# parser.add_argument("--N", type=int, default=20, help="The number of sampled examples")
# parser.add_argument("--delta", type=float, default=0.5, help="The balanced coefficient")
# parser.add_argument("--zeta", type=float, default=3.0, help="The upper bound of neighborhood")
# opt = parser.parse_args()

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

transforms = T.Compose(
    [T.Resize(299), T.ToTensor()]
)

def clip_by_tensor(t, t_min, t_max):
    """
    clip_by_tensor
    :param t: tensor
    :param t_min: min
    :param t_max: max
    :return: cliped tensor
    """
    result = (t >= t_min).float() * t + (t < t_min).float() * t_min
    result = (result <= t_max).float() * result + (result > t_max).float() * t_max
    return result

def save_image(images,names,output_dir):
    """save the adversarial images"""
    if os.path.exists(output_dir)==False:
        os.makedirs(output_dir)

    for i,name in enumerate(names):
        img = Image.fromarray(images[i].astype('uint8'))
        img.save(output_dir + name)

def PGN(images, gt, model, min, max):
    """
    The attack algorithm of our proposed CMI-FGSM
    :param images: the input images
    :param gt: ground-truth
    :param model: substitute model
    :param mix: the mix the clip operation 
    :param max: the max the clip operation
    :return: the adversarial images
    """
    eps = e / 255.0
    num_iter = 10
    alpha = eps / num_iter
    momentum = 1
    x = images.clone().detach().cuda()
    zeta = 3
    delta = 0.5
    N = 20
    grad = torch.zeros_like(x).detach().cuda()
    for i in range(num_iter):
        avg_grad = torch.zeros_like(x).detach().cuda()
        for _ in range(N):
            x_near = x + torch.rand_like(x).uniform_(-eps*zeta, eps*zeta)
            x_near = V(x_near, requires_grad = True)
            output_v3 = model(x_near)
            loss = F.cross_entropy(output_v3, gt)
            g1 = torch.autograd.grad(loss, x_near,
                                        retain_graph=False, create_graph=False)[0]
            x_star = x_near.detach() + alpha * (-g1)/torch.abs(g1).mean([1, 2, 3], keepdim=True)

            nes_x = x_star.detach()
            nes_x = V(nes_x, requires_grad = True)
            output_v3 = model(nes_x)
            loss = F.cross_entropy(output_v3, gt)
            g2 = torch.autograd.grad(loss, nes_x,
                                        retain_graph=False, create_graph=False)[0]

            avg_grad += (1-delta)*g1 + delta*g2
        noise = (avg_grad) / torch.abs(avg_grad).mean([1, 2, 3], keepdim=True)
        noise = momentum * grad + noise
        grad = noise
        
        x = x + alpha * torch.sign(noise)
        x = clip_by_tensor(x, min, max)
    return x.detach()



In [ ]:
import pickle

batch_size = 100
d = {}
for i , (name_v, model_v, weights_v) in enumerate(models): 
        victim = model_v(weights=weights_v.DEFAULT)
        print("______________________")
        print("Victim: "+ name_v)
    
        S = models.copy()
        S.pop(i)
        victim = model_with_normalization(victim, normalization)
        victim.eval()
        victim = victim.to(device)
        
        with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
            new_data_loader = pickle.load(f)
    
        for j , (name_g,model_g, weights_g) in enumerate(S):
                       row = [0,0,0]
                       print("g: "+ name_g) 
                       g = model_g(weights=weights_g.DEFAULT) 
                       Se = S.copy()
                       Se.pop(j)
                       g = model_with_normalization(g, normalization)
                       g.eval()
                       g = g.to(device)
                       ASR = 0
                       l2_score = 0
            
                       for batch , (images, labels) in enumerate(new_data_loader):
                           print(batch)
                           labels =labels.to(device)
                           #PGN
                           images_min = clip_by_tensor(images - e / 255.0, 0.0, 1.0)
                           images_max = clip_by_tensor(images + e / 255.0, 0.0, 1.0)
                           x_p = PGN(images, labels, g, images_min, images_max)
                           adv_label_p = victim(x_p).argmax(dim=1)
                           # adv_label_p = adv_label_p.to('cpu')
                           ASR += (labels != adv_label_p).int().sum().item()
                           l2_score += torch.norm(images - x_p)
                           #print(l2_score)
            
                       d[f'{name_v}_{name_g}_ASR'] = ASR/1000
                       d[f'{name_v}_{name_g}_L2'] = l2_score/1000
                       print(ASR)
                        

______________________
Victim: vit_b_16
g: efficientnet_b0
0
1
2
3
4
5
6
7
8
9
835
g: swin_s
0
1
2
3
4


In [ ]:
with open('ranking_simba_baselines/PGN.pkl', 'wb') as f:
     pickle.dump(d, f)

In [ ]:
#PGN
batch_size = 100
for i , (name_v, model_v, weights_v) in enumerate(models): 
    if i==3:
        victim = model_v(weights=weights_v.DEFAULT)
        print("______________________")
        print("Victim: "+ name_v)
    
        S = models.copy()
        S.pop(i)
        victim = model_with_normalization(victim, normalization)
        victim.eval()
        victim = victim.to(device)
        
        with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
            new_data_loader = pickle.load(f)
    
        # new_data_loader = DataLoader(
        #         new_data_loader.dataset[x:x+100], batch_size=batch_size, shuffle=True
        #     )
        
    
        for j , (name_g,model_g, weights_g) in enumerate(S):
                       row = [0,0,0]
                       print("g: "+ name_g) 
                       g = model_g(weights=weights_g.DEFAULT) 
                       Se = S.copy()
                       Se.pop(j)
                       g = model_with_normalization(g, normalization)
    
                       g.eval()
                       g = g.to(device)
                       ASR_total = 0
                       for batch , (images, labels) in enumerate(new_data_loader):
                           labels =labels.to(device)
                           perturbation_confidences = []
                           perturbation_success = []
    
                           augmentations_confidences = []
                           augmentations_success = []
    
                           augmentation_confidences = []
                           augmentation_success = [] 
    
                           confidences = []
    
                           print(batch)
    
                           
                           #PGN
                           images_min = clip_by_tensor(images - e / 255.0, 0.0, 1.0)
                           images_max = clip_by_tensor(images + e / 255.0, 0.0, 1.0)
                           x_p = PGN(images, labels, g, images_min, images_max)
                           adv_label_p = victim(x_p).argmax(dim=1)
                           # adv_label_p = adv_label_p.to('cpu')
                           ASR = (labels != adv_label_p).int().sum().item()
                           print(ASR)
                           row[0] += ASR
    
    
    
    
                           
                           for _ in tqdm(range(200)):
                               
                                # PGD_attack = PGD(g, eps=e/255, alpha=0.3/255, steps =20, random_start=True)
    
                               #PGN+ s2
                                
                                adv_label_p = victim(x_p).argmax(dim=1)
                                adv_label_p = adv_label_p.to('cpu')
                                # mixing augmentations
                                x_a = augmentations(images)
                                images_min = clip_by_tensor(x_a - e / 255.0, 0.0, 1.0)
                                images_max = clip_by_tensor(x_a + e / 255.0, 0.0, 1.0)
                                x_a = PGN(x_a, labels, g, images_min, images_max)
                                confidance = victim(x_a).softmax(dim=1)
                                conf = confidance[torch.arange(confidance.size(0)),labels].detach().cpu().numpy()
                                adv_label_a = confidance.argmax(dim=1)
                                # adv_label_a = adv_label_a.to('cpu')
                                del confidance
    
                                torch.cuda.empty_cache()
    
                                # average_confidences_p =  np.zeros(batch_size, dtype=np.float32)
                                average_confidences_a =  np.zeros(batch_size, dtype=np.float32)
                                # average_confidences_o =  np.zeros(batch_size, dtype=np.float32)
                                for n , model_s, weights_s in Se:
    
                                    model = model_s(weights=weights_s.DEFAULT)
                                    model = model.to(device)
                                    model = model_with_normalization(model, normalization)
                                    model.eval()
    
    
                                    outputs_a = model(x_a).softmax(dim=1)
                                    confidences_a = outputs_a[torch.arange(outputs_a.size(0)),labels].detach().cpu().numpy()                            
                                    average_confidences_a += confidences_a
                                    del outputs_a,confidences_a
    
                                    del model
    
    
                                    torch.cuda.empty_cache()
                                del x_a#x_p,x_a,x_o
    
    
    
                                # average_confidences_p /= len(Se)
                                average_confidences_a /= len(Se)
                                # average_confidences_o /= len(Se)
    
                                # equality_tensor_p = (labels != adv_label_p).int()
                                equality_tensor_a = (labels != adv_label_a).int() 
                                # equality_tensor_o = (labels != adv_label_o).int()
    
    
    
                                confidences.append(conf)
    
                                # perturbation_confidences.append(average_confidences_p)
                                # perturbation_success.append(equality_tensor_p)
                                augmentations_confidences.append(average_confidences_a)
                                augmentations_success.append(equality_tensor_a)
                                # augmentation_confidences.append(average_confidences_o)
                                # augmentation_success.append(equality_tensor_o)
    
                                torch.cuda.empty_cache()
                         
                           merged_results_a = [
                                                tuple(
                                                    zip(
                                                        np.array(augmentations_confidences)[:, i],
                                                        np.array(confidences)[:, i],
                                                        torch.stack(augmentations_success).cpu().numpy()[:, i]
                                                    )
                                                ) 
                                                for i in range(batch_size)
                                            ]
    
                           sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]
    
                           
    
                           best_result = 0
                           random_result = 0 
                           best_v = 0
                           for sublist in sorted_results_a:
    
                                best_result += sublist[0][2]
                                # best_v += min(sublist, key=lambda x: x[1])[2]
                                # Increment the best result based on the first item's binary value
                                #best_result += sublist[0][1]
                                # Calculate the sum of random items in each sublist
                                random_result += random.choice(sublist)[2]
    
    
                           row[1]+=random_result
                           row[2]+=best_result
                           # row[6]+=best_v
                           print(row) 
                           # upperB[(name_v,name_g)] += best_v  
                           # print(results_a)     
                           # print(upperB)  
                           del images, labels
                           torch.cuda.empty_cache()
                            
    
               
                       with open('PGN_all/'+name_v+'_'+name_g+'_'+str(e)+'.pkl', 'wb') as f:
                         pickle.dump(row, f)

In [ ]:
# import pickle 
# p=0
# c=0
# for i in range(10):
#     with open('SSA/efficientnet_b0_vit_b_16_25.5'+str(i*100)+'.pkl', 'rb') as f:
#         l = pickle.load(f)
#         c+=l[0]
#         p+=l[2]
# print(p)
# print(c)